# Entrenador de Gatos y Perros
Alan Badillo Salas (badillo.soft@hotmail.com)

Este mini-proyecto tiene la intención de crear una red neuronal de tipo CNN, para aprender a determinar si una imagen corresponde a un gato o a un perro.

Para más información sobre las redes CNN visita: https://medium.freecodecamp.org/an-intuitive-guide-to-convolutional-neural-networks-260c2de0a050

Lo primero que debemos hacer es descargar el archivo zip que contiene 10,000 imágenes de gatos y perros en una resolución de 32x32 pixeles a color. Cada imagen tiene como nombre su identificador. El archivo de imágenes lo puede descargar de http://badillosoft.com/train_cat_dog.zip.

Una vez descargado el archivo debemos descomprimirlo y todas las imágenes se encontrarán la carpeta `train_cat_dog`.

## Importaciones

In [1]:
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Activation, Dense, Dropout, BatchNormalization
from keras import optimizers, regularizers
import numpy as np
import pandas as pd

print("Se han importado las librerías necesarias para el análisis")

Using TensorFlow backend.


Se han importado las librerías necesarias para el análisis


Primero hay que cargar el `DataFrame` con los id's de las imágenes y sus etiquetas para utilizarlos en el proceso de aprendizaje.

In [2]:
train_cat_dog = pd.read_csv("http://badillosoft.com/train_cat_dog.csv")

print(train_cat_dog.info())
print(train_cat_dog.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
Unnamed: 0    10000 non-null int64
id            10000 non-null int64
label         10000 non-null object
dtypes: int64(2), object(1)
memory usage: 234.4+ KB
None
   Unnamed: 0  id label
0           9  10   cat
1          17  18   cat
2          21  22   cat
3          26  27   cat
4          27  28   dog


Definimos un generador de datos para que tome el `DataFrame` de pandas que contiene los id's de las imágenes y sus etiquetas (`train_cat_dog`). Observa que el generador de datos define que el 20% de ellos serán utilizados para la validación.

In [3]:
datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)

print(datagen)

Creamos el generador de entrenamiento, esto se refiere a un generador que va a cargar las imágenes desde nuestra carpeta basado en el `DataFrame` (`train_cat_dog`) para proveer el `x_train` y el `y_train`.

In [4]:
train_generator = datagen.flow_from_dataframe(
    dataframe=train_cat_dog,           # Dataframe de pandas con los id's de las imágnes y las etiquetas
    directory="./train_cat_dog",       # La ruta a la carpeta que contiene las imágenes de entrenamiento
    x_col="id",                        # La columna del dataframe para formar x_train, y_train
    y_col="label",                     # La columna del dataframe para formar x_test, y_test
    has_ext=False,                     # Indica si la columna x ya tiene la extensión de la imagen
    subset="training",                 # Indica el tipo de generador (training o validation)
    batch_size=32,                     # Indica el tamaño del bloque para las épocas
    seed=42,                           # Indica la semilla aleatoria
    shuffle=True,                      # Indica si las imágenes se cargarán aleatoriamente
    class_mode="categorical",          # Indica el tipo de aprendizaje (en este caso categoríco)
    target_size=(32, 32)               # Indica la resolución de las imágenes
)

Found 8000 images belonging to 2 classes.


De forma similar vamos a crear un generador de imágenes para las validaciones, esto creará los `x_test` y los `y_test`.

In [5]:
test_generator = datagen.flow_from_dataframe(
    dataframe=train_cat_dog,           # Dataframe de pandas con los id's de las imágnes y las etiquetas
    directory="./train_cat_dog",       # La ruta a la carpeta que contiene las imágenes de entrenamiento
    x_col="id",                        # La columna del dataframe para formar x_train, y_train
    y_col="label",                     # La columna del dataframe para formar x_test, y_test
    has_ext=False,                     # Indica si la columna x ya tiene la extensión de la imagen
    subset="validation",               # Indica el tipo de generador (training o validation)
    batch_size=32,                     # Indica el tamaño del bloque para las épocas
    seed=42,                           # Indica la semilla aleatoria
    shuffle=True,                      # Indica si las imágenes se cargarán aleatoriamente
    class_mode="categorical",          # Indica el tipo de aprendizaje (en este caso categoríco)
    target_size=(32, 32)               # Indica la resolución de las imágenes
)

Found 2000 images belonging to 2 classes.


Con los generadores anteriores, ya podemos alimentar nuestra red CNN. Por lo que, primero debemos crear la CNN como sigue.

In [6]:
model = Sequential()

# Capas Convolutivas (El filtro/kernel)
model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32, 32, 3)))
model.add(Activation("relu"))

#model.add(Conv2D(32, (3, 3)))
#model.add(Activation("relu"))

# Capas Pooling (El reductor)
model.add(MaxPooling2D(pool_size=(2, 2)))

# Capa Dropout (La pérdida)
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))

#model.add(Conv2D(64, (3, 3)))
#model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

# Capa Flatten (El aplanador)
model.add(Flatten())

# Capas de Clasificación (El aprendizaje)
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
__________

Ya que tenemos el modelo, podemos entrenarlo con los generadores `train_generator` y `test_generator`.

In [7]:
# Calculamos los tamaños de salto para los kernels
STEP_TRAIN = train_generator.n // train_generator.batch_size
STEP_TEST = test_generator.n // test_generator.batch_size

model.fit_generator(
    generator=train_generator,      # Indica quién genera las imágenes de entranamiento
    steps_per_epoch=STEP_TRAIN,     # Indica cuántos pasos se realizarán por época
    validation_data=test_generator, # Indica el generador de imágenes para la validación
    validation_steps=STEP_TEST,     # Indica cuántos pasos se realizarán para la validación
    epochs=10                       # Indica las épocas de entrenamiento
)

Epoch 1/10
250/250 [==============================] - 25s 100ms/step - loss: 0.6778 - acc: 0.5745 - val_loss: 0.6341 - val_acc: 0.6588
Epoch 2/10
250/250 [==============================] - 24s 97ms/step - loss: 0.6257 - acc: 0.6520 - val_loss: 0.5960 - val_acc: 0.6855
Epoch 3/10
250/250 [==============================] - 23s 93ms/step - loss: 0.5938 - acc: 0.6860 - val_loss: 0.5781 - val_acc: 0.7048
Epoch 4/10
250/250 [==============================] - 23s 94ms/step - loss: 0.5700 - acc: 0.7022 - val_loss: 0.5552 - val_acc: 0.7236
Epoch 5/10
250/250 [==============================] - 21s 84ms/step - loss: 0.5546 - acc: 0.7147 - val_loss: 0.5524 - val_acc: 0.7221
Epoch 6/10
250/250 [==============================] - 20s 82ms/step - loss: 0.5383 - acc: 0.7300 - val_loss: 0.5696 - val_acc: 0.6966
Epoch 7/10
250/250 [==============================] - 21s 83ms/step - loss: 0.5219 - acc: 0.7438 - val_loss: 0.5231 - val_acc: 0.7439
Epoch 8/10
250/250 [==============================] - 23s 90m

Ahora la red CNN ya está entrenada, por lo que medimos su desempeño con las imágenes de validación (`test_generator`)

In [8]:
metrics = model.evaluate_generator(generator=test_generator, steps=STEP_TEST)

print(metrics)

[0.49700924293781684, 0.7627032520325203]


## Probar el clasificador con imágenes reales

Hasta aquí ya hemos entrenado nuestro modelo para aprender a reconocer imágenes de gatos y perros, por lo que podemos ahora descargar una imagen desde internet, reducirla a una resolución de `32x32` pixeles y evaluarla en el modelo.

In [23]:
from PIL import Image
import requests
# Python 2
from StringIO import StringIO

# Gato
#url = "https://www.readersdigest.ca/wp-content/uploads/sites/14/2011/01/4-ways-cheer-up-depressed-cat.jpg"
# Perro
#url = "https://img.huffingtonpost.com/asset/5b7fdeab1900001d035028dc.jpeg?cache=sixpwrbb1s&ops=1910_1000"
# Gato
url = "https://amp.businessinsider.com/images/5654150584307663008b4ed8-750-563.jpg"
# Perro
#url = "https://www.guidedogs.org/wp-content/uploads/2018/01/Mobile.jpg"

# Python 2
response = requests.get(url)
# Python 3
# response = request.get(url, stream=True)

# Python 2
im = Image.open(StringIO(response.content))
# Python 3
# im = Image.open(response.raw)

print(im)

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=750x563 at 0x11C1BBD90>


Ahora hay que reducir la imagen a `32x32` pixeles.

In [24]:
im = im.resize((32, 32), Image.NEAREST)

print(im)

<PIL.Image.Image image mode=RGB size=32x32 at 0x11C07DBD0>


Ahora necesitamos convertir la imagen en un arreglo de números `(32, 32, 3)` y convertirlo a una matriz para que se pueda hacer la predicción. Recordemos que `model.predict` espera una matriz de muestras para hacer la predicción en todas las muestras (no podemos manderle sólo un ejemplo para la predicción).

In [25]:
im_array = np.asarray(im)

x_test = np.array([im_array]) # np.array([im1, im2, im3, ...])

print(x_test)

[[[[202 188 175]
   [198 188 176]
   [199 187 175]
   ...
   [202 170 121]
   [136 106  68]
   [ 37  35  23]]

  [[198 184 171]
   [198 184 173]
   [195 183 171]
   ...
   [204 170 124]
   [143 113  75]
   [ 29  26  17]]

  [[197 183 170]
   [195 181 168]
   [193 181 167]
   ...
   [205 170 128]
   [150 117  86]
   [ 24  21  14]]

  ...

  [[ 62  45  29]
   [ 54  38  15]
   [ 53  40  21]
   ...
   [189 157 134]
   [208 176 165]
   [204 171 162]]

  [[ 51  35  19]
   [ 54  39  18]
   [116  87  53]
   ...
   [153 119  91]
   [199 166 157]
   [203 169 160]]

  [[ 40  32  13]
   [ 77  60  34]
   [189 155 117]
   ...
   [115  84  56]
   [201 173 161]
   [218 188 178]]]]


Ahora ya podemos hacer la predicción sobre `x_test`. El resultado nos va a indicar un vector con la probabilidad de que pertenezca a las distintas clases.

In [26]:
predict = model.predict(x_test)

print(predict)

[[1. 0.]]


En este caso tenemos dos clases y nos está indicando que tiene una probabilidad `0` de pertenecer a la primer clase y una probabilidad de `1` de pertenecer a la segunda clase. Para ver las clases hacemos.

In [27]:
print(train_generator.class_indices)

{'dog': 1, 'cat': 0}


Ahora vamos a tomar las clases y los resultados de la predicción para que imprima que combine los scores.

In [28]:
class_indices = train_generator.class_indices
for label in class_indices:
    for score in predict:
        index = class_indices[label]
        print("{}: {}".format(label, score[index]))

dog: 0.0
cat: 1.0


## Guardar el modelo y entrenamiento

Ahora que ya todo está listo podemos guardar el modelo y su entrenamiento para poder utilizarlo en otras aplicaciones como se verá más adelante.

In [29]:
import json

with open("cat_dog_classes.json", "w") as f:
    f.write(json.dumps(class_indices))

with open("cat_dog_model.json", "w") as f:
    f.write(model.to_json())

model.save("cat_dog_weights.h5")

## Exponer una interfaz web y servicios web (API-REST)

Finalmente podemos generar mediante flask un servidor que reciba la url de una imagen y nos diga si es perro o si es gato. Así como proveer una página web con una caja y botón que nos diga sobre una url si es gato o si es perro.

In [31]:
from flask import Flask, request
import numpy as np
from PIL import Image
import requests
# Python 2
from StringIO import StringIO
from keras.models import model_from_json
import json

with open("cat_dog_model.json") as f:
    model = model_from_json(f.read())

model.load_weights("cat_dog_weights.h5")

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

with open("cat_dog_classes.json") as f:
    class_indices = json.load(f)

def cat_dog_predict(url):
    response = requests.get(url)
    im = Image.open(StringIO(response.content))
    im = im.resize((32, 32), resample=Image.NEAREST)
    im_vec = np.asarray(im)
    x_test = np.array([im_vec])
    predict = model.predict(x_test)[0]
    score = {}
    for cat in class_indices:
        index = class_indices[cat]
        score[cat] = predict[index]
    return score

#print(cat_dog_predict("https://www.guidedogs.org/wp-content/uploads/2018/01/Mobile.jpg"))

app = Flask(__name__)

html = '''
<form action="/api/cat_dog" method="get">
    <input id="url" name="url" placeholder="url de gato o perro" >
</form>
'''

@app.route("/")
def home():
    return html

@app.route("/api/cat_dog")
def cat_dog():
    url = request.args["url"]
    score = cat_dog_predict(url)
    result = '<img src="{}" width="200" >'.format(url)
    for cat in score:
        result += "<h1>{}: {}</h1>".format(cat, score[cat])
    result += html
    return result

app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Dec/2018 00:18:09] "GET /api/cat_dog?url=https%3A%2F%2Fimages.theconversation.com%2Ffiles%2F205966%2Foriginal%2Ffile-20180212-58348-7huv6f.jpeg%3Fixlib%3Drb-1.1.0%26q%3D45%26auto%3Dformat%26w%3D926%26fit%3Dclip HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2018 00:18:10] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Dec/2018 00:18:52] "GET /api/cat_dog?url=https%3A%2F%2Fencrypted-tbn0.gstatic.com%2Fimages%3Fq%3Dtbn%3AANd9GcRWB6W7GLsVLLDYp72NtIGnB4r1aJVpVnOed17IB2abKLY_8tAl HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2018 00:18:53] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Dec/2018 00:19:15] "GET /api/cat_dog?url=https%3A%2F%2Fwww.gannett-cdn.com%2F-mm-%2F4a9f3ddc8f1dae539e5301fb8821077a1ffacfdc%2Fc%3D99-0-401-170%2Flocal%2F-%2Fmedia%2F2018%2F05%2F14%2FUSATODAY%2Fusatsports%2Fwp-USAT-allthemoms-front1-17730-cat.png%3Fwidth%3D580%26height%3D326%26fit%3Dcrop HTTP/1.1" 200 -
127.0.0.1 - - [02/Dec/2018 00:19:15]